In [4]:
import psycopg2
import csv
import os
#from dotenv import load_dotenv

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname='patientdb',  # Your PostgreSQL database name
    user='projectuser',  # Your PostgreSQL user
    password='projectpassword',  # Your PostgreSQL password
    host='db',  # Host for PostgreSQL
    port='5432'  # PostgreSQL port
)

cursor = conn.cursor()

# Query to extract data from PostgreSQL
query = "SELECT * FROM insurance_plans"
cursor.execute(query)

# Fetch all rows
rows = cursor.fetchall()

# Write to a CSV file
with open("insurance_plans.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([desc[0] for desc in cursor.description])  # Write header
    writer.writerows(rows)

cursor.close()
conn.close()


OperationalError: connection to server at "db" (10.110.132.232), port 5432 failed: Operation timed out
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "db" (155.33.18.75), port 5432 failed: Operation timed out
	Is the server running on that host and accepting TCP/IP connections?


In [3]:
import snowflake.connector

# Connect to Snowflake
conn = snowflake.connector.connect(
    user='BISON',  # Snowflake username
    password='Anoushka1106!',  # Snowflake password
    account='PDB57018',  # Your Snowflake account identifier
    warehouse='HEALTHCARE',  # Your Snowflake warehouse
    database='HEALTHCARE_INSURANCE_DB',  # Your Snowflake database
    schema='PLAN_SCHEMA'  # Your Snowflake schema
)

cursor = conn.cursor()

# Create a stage in Snowflake (if not created already)
cursor.execute("""
    CREATE STAGE my_stage 
    FILE_FORMAT = (TYPE = 'CSV', FIELD_OPTIONALLY_ENCLOSED_BY = '"', FIELD_DELIMITER = ',')
""")

# Upload CSV file to Snowflake stage
cursor.execute("""
    PUT file://insurance_plans.csv @my_stage
""")

# Load data from the stage into the table
cursor.execute("""
    COPY INTO PLAN_SCHEMA.INSURANCE_PLANS
    FROM @my_stage/insurance_plans.csv
    FILE_FORMAT = (TYPE = 'CSV', FIELD_OPTIONALLY_ENCLOSED_BY = '"', FIELD_DELIMITER = ',')
    ON_ERROR = 'CONTINUE'
""")

cursor.close()
conn.close()


ModuleNotFoundError: No module named 'snowflake'